# Analysis of a Book Reading Service Database



The world was taken by surprise by the coronavirus, changing the usual order of things. People no longer go out, visit cafes, or shopping centers in their free time. However, there is now more time for books.

Objective: Analyze the database of a large book reading subscription service. It contains information about books, publishers, authors, and user reviews of books. This data will help formulate a value proposition for a new product.

**Data Description**

**Table: `books`**

Contains data about books:

- `book_id` - book identifier;
- `author_id` - author identifier;
- `title` - book title;
- `num_pages` - number of pages;
- `publication_date` - book publication date;
- `publisher_id` - publisher identifier.

**Table: `authors`**

Contains data about authors:

- `author_id` - author identifier;
- `author` - author's name.

**Table: `publishers`**

Contains data about publishers:

- `publisher_id` - publisher identifier;
- `publisher` - publisher's name;

**Table: `ratings`**

Contains data about user ratings of books:

- `rating_id` - rating identifier;
- `book_id` - book identifier;
- `username` - username of the user who left the rating;
- `rating` - book rating.

**Table: `reviews`**

Contains data about user reviews of books:

- `review_id` - review identifier;
- `book_id` - book identifier;
- `username` - username of the user who wrote the review;
- `text` - review text.

**Tasks**

- Calculate the number of books released after January 1, 2000;
- For each book, calculate the number of reviews and the average rating;
- Determine the publisher that has released the most books with more than 50 pages;
- Identify the author with the highest average rating for books, considering only books with 50 or more ratings;
- Calculate the average number of reviews from users who have given more than 48 ratings.

## Data Loading and Analysis

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
query = '''SELECT COUNT(DISTINCT review_id) FROM reviews'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


## Number of Books Released After January 1, 2000

In [7]:
query = """SELECT COUNT(book_id) 
           FROM books
           WHERE publication_date > '2000-01-01'"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)


,count
0,819


819 books were released after January 1, 2000.

## Number of Reviews and Average Rating

In [47]:
query = """SELECT b.title,
                  COUNT(DISTINCT rw.review_id) AS count_review,
                  AVG(r.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           GROUP BY b.book_id
           ORDER BY avg_rating DESC, count_review DESC"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)


,title,count_review,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


As a result, we generated a table that shows the number of reviews and the average rating for each book.

## Publisher with the Most Books over 50 Pages

In [9]:
query = """SELECT DISTINCT publisher,
                  COUNT(book_id) OVER (PARTITION BY publisher_id) as quantity
           FROM books as b
           LEFT JOIN publishers as p USING(publisher_id)
           WHERE b.num_pages > 50
           ORDER BY quantity DESC
           LIMIT 1"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,publisher,quantity
0,Penguin Books,42



Penguin Books has released the highest number of books - 42.

## Author with the Highest Average Rating (50 or More Ratings)

In [10]:
query = """WITH temp AS (SELECT *,
                  COUNT(rating_id) OVER (PARTITION BY book_id) as quantity
           FROM ratings)
           SELECT author,
                  ROUND(AVG(rating) OVER (PARTITION BY author_id), 1) as avg_rating
           FROM temp
           JOIN books as b USING(book_id)
           JOIN author as a USING(author_id)
           WHERE quantity >= 50
           ORDER BY avg_rating DESC
           LIMIT 1"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.3



The book with the highest average rating is by J.K. Rowling and Mary GrandPré, with a rating of 4.3.

## Average Number of Reviews from Users Who Have Given More Than 48 Ratings

In [11]:
query = """SELECT AVG(q_reviews)
           FROM (SELECT username, COUNT(*) q_reviews
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(*) > 48)
           GROUP BY username) r"""

con = engine.connect()

pd.io.sql.read_sql(sql=text(query), con=con)

,avg
0,24.0



The average number of reviews from users who have given more than 48 ratings is 24.

## Conclusion



Based on the analysis, we have determined the following:
- 819 books were released after January 1, 2000.
- Penguin Books has released the highest number of books - 42.
- The book by J.K. Rowling and Mary GrandPré has the highest average rating of 4.3.
- The average number of reviews from users who have given more than 48 ratings is 24.
- We have also supplemented the table with the number of reviews and average rating for each book.
